<a href="https://colab.research.google.com/github/neonco/Jupiter-Notebooks/blob/master/task25_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задания 25. Маски

В соответствии со спецификацией ФИПИ, 25 задание относится к заданиям высокого уровня сложности. На её решение отведено 20 минут. Учитывая совсем небольшой объём кода, такие временные рамки обусловлены переборной сутью алгоритма.
Оценка сложности и подбор оптимального метода решения позволят решать задачи намного быстрее.

### Задача 1. Демоверсия ФИПИ 2025

Назовём маской числа последовательность цифр, в которой также могут
встречаться следующие символы:
- символ «?» означает ровно одну произвольную цифру;
- символ «\*» означает любую последовательность цифр произвольной
длины; в том числе «\*» может задавать и пустую последовательность.
Например, маске 123*4?5 соответствуют числа 123405 и 12300405.

Среди натуральных чисел, не превышающих 10**10, найдите все числа,
соответствующие маске 3?12?14*5, делящиеся на 1917 без остатка.
В ответе запишите в первом столбце таблицы все найденные числа
в порядке возрастания, а во втором столбце – соответствующие им
результаты деления этих чисел на 1917.
Количество строк в таблице для ответа избыточно.



---


Решим задачу шаблонным **fnmatch**.
Эта встроенная функция из одноимённого модуля сопоставляет строку и маску, возвращая только True или False. Удобно то, что шаблон-маска записывается абсолютно также, как в задании.

Проверка строк через fnmatch занимает больше времени, чем проверка арифметических выражений с числами. Поэтому сначала проверим делимость, только после используем fnmatch.

In [ ]:
from fnmatch import fnmatch

mask = '3?12?14*5'

for n in range(1, 10**10):
    if n % 1917 == 0:
        if fnmatch(str(n), mask):
            print(n, n // 1917)

Программа написана верно и выдаёт правильные ответы, но делает это непростительно долго.
**Миллиард** - разумное ограничение перебора чисел для Python.
В задаче перебирается 10 миллиардов чисел, что вынуждает искать пути оптимизации.

#### Оптимизация проверки на делимость.
Возьмём число, заведомо делящееся на 1917. В случае натуральных чисел само число 1917 и будет первым таковым. Но удобнее использовать **0**, так как он делится на любое натуральное число. Будем брать не каждое следующее за нулём число, а с **шагом** в 1917.
Таким образом можно убрать проверку на делимость.


In [ ]:
from fnmatch import fnmatch

mask = '3?12?14*5'

for n in range(0, 10**10, 1917):
    if fnmatch(str(n), mask):
        print(n, n // 1917)

351261495 183235
3212614035 1675855
3412614645 1780185
3712414275 1936575
3912414885 2040905


Задача решилась за 10 секунд, так как перебор сократился в 1917 раз.

### Задача 2.
[ссылка на задание](https://education.yandex.ru/ege/collections/da3db128-d559-4f5b-b808-270108cda5c2/task/5)

Требуется найти все натуральные числа меньшие 10⁹, которые кратны 183 и соответствуют маске «??287*139».

Применив оценку 10**9 / 183, понимаем что укладываемся в миллиард. Решим аналогично предыдущей задаче.

In [1]:
from fnmatch import fnmatch

mask = '??287*139'

for n in range(0, 10**9, 183):
    if fnmatch(str(n), mask):
        print(n, n // 183)

142874139 780733
452876139 2474733
592871139 3239733
762878139 4168733
902873139 4933733


### Задача 3.
[ссылка на задание](https://education.yandex.ru/ege/collections/da3db128-d559-4f5b-b808-270108cda5c2/task/3)

Среди натуральных чисел, не превышающих 10**12 , найдите все числа, соответствующие маске 4?8?15?16?23, делящиеся на 123 с остатком 42.

Задача отличается от предыдущих требованием по делимости. При делении 0 на 123 остаток будет равен 0. Сдвинем начальную точку отсчёта до 42, выполнив условие остатка 42 при делении на 123. Шаг не меняется.

In [ ]:
from fnmatch import fnmatch

mask = '4?8?15?16?23'

for n in range(42, 10**12, 123):
    if fnmatch(str(n), mask):
        print(n, n // 123)

Программа работает слишком долго.
Оценим сложность как 10**12 / 123, что чуть меньше 10 миллиардов. Необходимо оптимизировать.

#### Оптимизация перебора.
Внимательно рассмотрим маску "4?8?15?16?23". В ней содержатся 8 цифр и 4 вопросительных знака, которые означают ровно одну произвольную цифру. Таким образом любое подходящее число должно быть минимум 12-тизначным. А если учитывать что верхняя граница перебора - 1000 миллиардов, то все эти числа не будут превосходить 12 цифр в записи.
Но даже такой перебор будет для нас долгим.

Остаётся только **генерировать** числа по маске.
Заметив что в маске стоят 4 вопросительных знака (не в крайней левой позиции), каждый из которых может быть одной из 10 цифр. Тогда перебирая все возможные варианты, получим их число 10\*10\*10\*10 = 10**4 = 10000.

Сгенерируем все эти варианты как **f-строки** и запишем в список как **числа**.

In [ ]:
m = []

digits = '0123456789'
for d1 in digits:
    for d2 in digits:
        for d3 in digits:
            for d4 in digits:
                n = f'4{d1}8{d2}15{d3}16{d4}23'
                m.append(int(n))

print(len(m), m[:5])


10000 [408015016023, 408015016123, 408015016223, 408015016323, 408015016423]


В задаче требуется найти количество чисел и максимальное из них.
Удобнее воспользоваться **генераторами списков** для фильтрации по условию делимости.

In [ ]:
res = [n for n in m if n % 123 == 42]
print(len(res), max(res))

85 498915816123


Или более стандартно через **for** и **переменные**.

In [ ]:
count = 0
max_n = 0
for n in m:
    if n % 123 == 42:
        count += 1
        max_n = max(max_n, n)
print(count, max_n)

85 498915816123
